# Long Short Time Memory

Table of contents
1. [Library imports](#Library-imports)
2. [Data Reading](#Data-reading)
    1. [Data cleaning](#Data-cleaning)
    2. [Data normalizarion](#Data-normalization)
3. [LSTM](#LSTM-models-fitting)
    1. [Fine tuning](#Fine-tuning)
    2. [Final model](#Final-model)
    
## Library imports

In [7]:
%pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from utils.utils import mix_data, generate_windows # custom functions

import tensorflow as tf                            # LSTM                              # DataFrames
import numpy as np                                 # Vectorial computation
import itertools                                   # Combinatory
import datetime                                    # Dates formatting

ModuleNotFoundError: No module named 'utils'

## Data reading

In [ ]:
df = pd.read_csv('../data/radon-data.csv')   # CSV reading
df.head()
df.size()

NameError: name 'pd' is not defined

### Data cleaning

In [ ]:
# Now we will parse the data, so it is usable
df.time = pd.to_datetime(df['time'], unit='s', origin='unix')      # date parse
df = df.drop(columns = ["time", "id", "sensor_id", "state_time"])  # drop useless columns
df.state = (df.state == "On").astype(int)                          # binarize state
df.head()

,radon,temperature,humidity,pressure,tvoc,state
0,202,25,50,1015,0,0
1,258,25,51,1015,0,1
2,202,24,51,1015,0,0
3,182,24,51,1015,0,0
4,189,24,51,1015,0,0


In [9]:
# we now select the data from the summer (longest clean data)
df_summer = df.iloc[36000:48000]
df_summer.head()

NameError: name 'df' is not defined

### Data normalization

In [ ]:
summer_min = df_summer.min() # needed to de-normalize data
summer_max = df_summer.max()

df_summer_normalized = (df_summer - summer_min) / (summer_max - summer_min)
df_summer_normalized.head()

,radon,temperature,humidity,pressure,tvoc,state
36000,0.119640,0.714286,0.666667,0.5,0.003463,1.0
36001,0.117911,0.714286,0.666667,0.5,0.001732,1.0
36002,0.123098,0.714286,0.666667,0.5,0.005195,1.0
36003,0.119640,0.714286,0.666667,0.5,0.006926,1.0
36004,0.119640,0.714286,0.666667,0.5,0.008658,1.0


## LSTM models fitting

In [ ]:
# We will use early stopping to select the best model
callback = tf.keras.callbacks.EarlyStopping(
    monitor              = 'val_loss',
    patience             = 30,
    restore_best_weights = True
)

# percentage of train samples
train_pct = 0.9

# We will store all the results in a data frame
results = pd.DataFrame(
    {i: [None]*15 for i in [1, 5, 10, 15, 25]}
)

cnt = 0 # helps counting

for number_of_covariates in range(4):
    for covariate_combination in itertools.combinations(("state", "humidity", "pressure", "tvoc"), 
                                                        number_of_covariates):
        print(f"EXECUTING OVER: {covariate_combination}".center(100, "*"))
        # We compute for each window size
        for window_size in [1, 5, 10, 15, 25]:
            print("\n")
            print(f"EXECUTING {window_size} WINDOW SIZE".center(100), "-")
            print("\n")
            windows = {i: generate_windows(df_summer_normalized[i], window_size, 1)
                        for i in ["radon"] + list(covariate_combination) }
            
            print("Windows computed, now building the model")
            
            data = mix_data(windows)
            N_total          = data.shape[0]
            test             = data[int(train_pct * N_total):, :, :]
            data             = data[:int(train_pct * N_total), :, :]
            
            model = tf.keras.models.Sequential(
                [
                    tf.keras.layers.InputLayer((window_size, len(windows.keys()))),
                    tf.keras.layers.LSTM(window_size * 2),
                    tf.keras.layers.Dense(16, activation = "relu"),
                    tf.keras.layers.Dense(16, activation = "relu"),
                    tf.keras.layers.Dense(1)
                ],
            )
            
            model.compile(
                loss = tf.keras.losses.MeanSquaredError(),
                metrics = tf.keras.metrics.RootMeanSquaredError(),
                optimizer = tf.keras.optimizers.RMSprop()
            )
            
            model.summary()
            
            model.fit(
                x                = data[:, :window_size, :],
                y                = data[:, window_size:, 0],
                epochs           = 1_000,
                shuffle          = False,
                validation_split = 0.2, 
                callbacks        = [callback,],
                verbose          = 0,
            )
            
            # Model trained
            
            # Now compute the test predictions and add them to the data frame
            rescale    = lambda x: x * (summer_max.radon - summer_min.radon) + summer_min.radon
            prediction = rescale( model.predict(test[:, :window_size, :]) )
            real       = rescale( test[:, window_size:, 0] )
            
            rmse_test = ( sum((prediction - real) ** 2) / len(prediction) ) ** (1/2)
            results[window_size][cnt] = rmse_test 
            print(f"Window {window_size}, covariates {covariate_combination}: {results[window_size][cnt]}")
        cnt += 1    
        print("*" * 100, "\n"*2)

*****************************************EXECUTING OVER: ()*****************************************


                                      EXECUTING 1 WINDOW SIZE                                        -


Windows computed, now building the model
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 2)                 32        
                                                                 
 dense (Dense)               (None, 16)                48        
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 369
Trainable params: 369
Non-trainable params: 0
_______

NameError: name 'df' is not defined

### Fine tuning
Once we know what model behaves the best, we try to explore this configuration.

In [ ]:
# We will use early stopping to select the best model
callback = tf.keras.callbacks.EarlyStopping(
    monitor              = 'val_loss',
    patience             = 30,
    restore_best_weights = True
)

# percentage of train samples
train_pct = 0.9

# We will store all the results in a data frame
results = pd.DataFrame(
    {i: [None]*4 for i in [4, 8, 16, 32]}
)

# dense configurations:
dense_combination_config = [[16], [16, 16], [32], [32, 32]]

window_size = 15

# we do not need to create the data in a loop:
windows = {i: generate_windows(df_summer_normalized[i], window_size, 1)
                for i in ["radon", "state"] }
data    = mix_data(windows)
N_total = data.shape[0]
test    = data[int(train_pct * N_total):, :, :]
data    = data[:int(train_pct * N_total), :, :]

for idx, dense_config in enumerate(dense_combination_config):
    for num_hidden_units in [4, 8, 16, 32]:
        
        model = tf.keras.models.Sequential(
            [
                tf.keras.layers.InputLayer((window_size, len(windows.keys()))),
                tf.keras.layers.LSTM(num_hidden_units),
                *[tf.keras.layers.Dense(i, activation = "relu") for i in dense_config],
                tf.keras.layers.Dense(1)
            ],
        )

        model.compile(
            loss = tf.keras.losses.MeanSquaredError(),
            metrics = tf.keras.metrics.RootMeanSquaredError(),
            optimizer = tf.keras.optimizers.RMSprop()
        )

        model.summary()

        model.fit(
            x                = data[:, :window_size, :],
            y                = data[:, window_size:, 0],
            epochs           = 1_000,
            shuffle          = False,
            validation_split = 0.2, 
            callbacks        = [callback,],
            verbose          = 0,
        )

        # Model trained

        # Now compute the test predictions and add them to the data frame
        rescale    = lambda x: x * (summer_max.radon - summer_min.radon) + summer_min.radon
        prediction = rescale( model.predict(test[:, :window_size, :]) )
        real       = rescale( test[:, window_size:, 0] )

        rmse_test = ( sum((prediction - real) ** 2) / len(prediction) ) ** (1/2)
        results[num_hidden_units][idx] = rmse_test 
        print(f"Config {idx} with {num_hidden_units} hidden units -> {rmse_test}")
        print("*" * 100, "\n"*2)

Model: "sequential_75"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_75 (LSTM)              (None, 4)                 112       
                                                                 
 dense_225 (Dense)           (None, 16)                80        
                                                                 
 dense_226 (Dense)           (None, 1)                 17        
                                                                 
Total params: 209
Trainable params: 209
Non-trainable params: 0
_________________________________________________________________
38/38 [==============================] - 1s 3ms/step
Config 0 with 4 hidden units -> [14.90602804]
**************************************************************************************************** 


Model: "sequential_76"
_________________________________________________________________
 Layer (type)              

## Final model

In [ ]:
# We will use early stopping to select the best model
callback = tf.keras.callbacks.EarlyStopping(
    monitor              = 'val_loss',
    patience             = 30,
    restore_best_weights = True
)

# percentage of train samples
train_pct = 0.9

window_size = 15

# we do not need to create the data in a loop:
windows = {i: generate_windows(df_summer_normalized[i], window_size, 1)
                for i in ["radon", "state"] }
data    = mix_data(windows)
N_total = data.shape[0]
test    = data[int(train_pct * N_total):, :, :]
data    = data[:int(train_pct * N_total), :, :]

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.InputLayer((window_size, len(windows.keys()))),
        tf.keras.layers.LSTM(4),
        tf.keras.layers.Dense(32, activation = "relu"),
        tf.keras.layers.Dense(1)
    ],
)

model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    metrics = tf.keras.metrics.RootMeanSquaredError(),
    optimizer = tf.keras.optimizers.RMSprop()
)

model.summary()

model.fit(
    x                = data[:, :window_size, :],
    y                = data[:, window_size:, 0],
    epochs           = 1_000,
    shuffle          = False,
    validation_split = 0.2, 
    callbacks        = [callback,],
)

# Model trained

# Now compute the test predictions and add them to the data frame
rescale    = lambda x: x * (summer_max.radon - summer_min.radon) + summer_min.radon
prediction = rescale( model.predict(test[:, :window_size, :]) )
real       = rescale( test[:, window_size:, 0] )

rmse_test = ( sum((prediction - real) ** 2) / len(prediction) ) ** (1/2)
print(f"RMSE -> {rmse_test}")

In [ ]:
model.save("../output/radon.h5", save_format = "h5")

In [ ]:
model.save("../output/radon.pb", save_format = "tf")

In [ ]:
summer_min, summer_max

In [ ]:
pd.DataFrame(
    {
        "predictions": prediction.flatten(),
        "real": real.flatten(),
    }
).to_csv("../data/prediction1s.csv", index = False)